In [4]:
from dotenv import load_dotenv, find_dotenv
import os
_=load_dotenv(find_dotenv())
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
)

d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:355: UserWarning: WARNING! max_completion_tokens is not default parameter.
                    max_completion_tokens was transferred to model_kwargs.
                    Please confirm that max_completion_tokens is what you intended.
  warnings.warn(
d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:355: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [7]:
# chatprompt template and str output parser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [8]:
chain= prompt | model | output_parser
chain.invoke({"soccer_player": "Lionel Messi"})

"Here's one:\n\nLionel Messi has a rare medical condition called Growth Hormone Deficiency (GHD). As a child, his growth was stunted due to a hormone deficiency, and he was only 4 ft 7 in (140 cm) tall at the age of 11. His family couldn't afford the treatment, but Carles Rexach, the sporting director of FC Barcelona, was so impressed by Messi's talent that he offered to pay for the treatment if Messi joined the club's youth academy. Messi accepted the offer and began his treatment, which involved daily injections of growth hormone. He eventually grew to be 5 ft 7 in (170 cm) tall!"

Use of .bind() to add arguments to a Runnable in a LCEL Chain


In [11]:
chain= prompt | model.bind(stop=["Messi"]) | output_parser
chain.invoke({"soccer_player": "Lionel Messi"})

"Here's one:\n\nLionel "

Combining LCEL Chains


In [12]:
#Combining LCEL Chains
prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()
chain.invoke({"politician": "Barack Obama"})

'Barack Obama was the first African American to serve as President of the United States, holding the office from 2009 to 2017.'

Combined chain


In [14]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()
composed_chain.invoke({"politician": "Barack Obama"})

"What a great question!\n\nYes, Barack Obama's presidency was a positive milestone for humanity in many ways. Here are some reasons why:\n\n1. **Breaking racial barriers**: Obama's election marked a historic moment in the fight against racial discrimination. He was the first African American to hold the office of President, shattering a centuries-old racial barrier. This achievement inspired hope and pride among people of color and symbolized progress towards equality.\n2. **Symbolic significance**: Obama's presidency showed that, in the United States, a country with a complex and often painful racial history, a person of color could hold the highest office. This sent a powerful message about the American Dream being accessible to all, regardless of race or ethnicity.\n3. **Inspirational impact**: Obama's presidency inspired a new generation of young people, particularly from minority backgrounds, to engage in politics and pursue careers in public service. His election and re-election 

Another example: a chain inside another chain


In [16]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)
chain1 = prompt1 | model | StrOutputParser()
chain2=(
    {"country": chain1,"language": itemgetter("language")}
    | prompt2 
    | model
    | StrOutputParser()
)
chain2.invoke({"politician": "Barack Obama", "language": "bangla"})

"আপনি কিছু বিভ্রান্তি করছেন! বারাক ওবামা সংযুক্ত রাষ্ট্রের রাষ্ট্রপতি এবং তিনি যুক্তরাষ্ট্রের হাওয়াইতে জন্মগ্রহণ করেছেন। তার মা আন ডানহাম আমেরিকান এবং বাবা বারাক ওবামা সিনিয়র কেনিয়ান।\n\nঅতএব, আপনার প্রশ্নের উত্তর হিসাবে, বারাক ওবামা সংযুক্ত রাষ্ট্রের নাগরিক এবং তিনি হাওয়াইতে জন্মগ্রহণ করেছেন। তিনি হাওয়াইতে বড় হয়েছেন এবং পরে ইলিনয়ে থাকতেন, যেখানে তিনি একজন সম্প্রদায় সংগঠক, রাজ্য সভা এবং সিনেটর হিসাবে কাজ করেছেন। \n\n(Translation: You're thinking of some confusion! Barack Obama is the President of the United States and he was born in Hawaii, United States. His mother Ann Dunham is American and his father Barack Obama Sr. is Kenyan.\n\nSo, to answer your question, Barack Obama is a citizen of the United States and he was born in Hawaii. He grew up in Hawaii and later lived in Illinois, where he worked as a community organizer, state senator, and Senator.)"